In [1]:
!pip install torchinfo

In [2]:
import os
import torch
import pandas as pd
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt

# import torchvision.transforms.functional
# import torchvision.transforms.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn

from torchinfo import summary
from tqdm import tqdm

### Initialization

In [ ]:
path = "/Users/annahalloy/Documents/EPFL/MA1/IPEO/dataset_amazon"

In [3]:
from Multilabel_Amazon_Engine import checking_folder, train, show_4_image_in_batch, batch_prediction
from Multilabel_Amazon_Module import AmazonSpaces, MultiLayerCNN, AdjustSaturation

Enter the data folder path:  /Users/annahalloy/Documents/EPFL/MA1/IPEO/dataset_amazon


In [4]:
data_folder = '../IPEO_Planet_project'
if not os.path.exists(data_folder):
    data_folder = input("Enter the data folder path: ")
    assert os.path.exists(data_folder), "I did not find the folder at, "+str(data_folder)

Enter the data folder path:  /Users/annahalloy/Documents/EPFL/MA1/IPEO/dataset_amazon


### model on gpu

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [6]:
model = MultiLayerCNN().to(device)

### Load different datasets

In [7]:
train_csv = 'training.csv'
validation_csv = 'validation.csv'
test_csv = 'test.csv'

Transform_choice = transforms.Compose([transforms.ToTensor(), transforms.CenterCrop(256),transforms.RandomAutocontrast(p=1), AdjustSaturation(1.2)])

train_dataset = AmazonSpaces(csv_file=train_csv,
                                    root_dir=f'{data_folder}/train-jpg', transform=Transform_choice)
validation_dataset = AmazonSpaces(csv_file=validation_csv,
                                    root_dir=f'{data_folder}/train-jpg', transform=Transform_choice)
test_dataset = AmazonSpaces(csv_file=test_csv,
                                    root_dir=f'{data_folder}/train-jpg', transform=Transform_choice)

### Dataloader, criterion, optimizer

In [8]:
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size,drop_last = True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size,drop_last = True)
### ON TOUCHE PAS CA
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True)

In [9]:
criterion = nn.BCEWithLogitsLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.01)

### Define a single training step

In [10]:
weight_value_before = float(model.fc.weight[0,0])
print(weight_value_before)

-0.006136296316981316


In [11]:
# lets take a single batch manually (next(enumerate(iterator)) just takes one element at a time)
idx, batch = next(enumerate(train_dataloader))

model.train() # set model to training mode
optim.zero_grad() # make sure gradients are all zeroed

# retrieve image and label from the batch
x, y = batch['image'], batch['labels']

device = "cpu" # specify to run this operation on GPU (otherwise write "cpu")

# move model and data to device
model = model.to(device)
x = x.to(device)
y = y.to(device)

# TODO: make forward pass (i.e., prediction) of x with the model
y_prediction = model(x)

# TODO: calculate the loss
loss = criterion(y_prediction, y)

# backpropoagation of gradients
loss.backward() 

# update model parameters
optim.step()

In [12]:
loss

tensor(0.6772, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [13]:
# let's print the value of a single weight after the training step
weight_value_after = float(model.fc.weight[0,0])
print(weight_value_after)

# we can also print the gradient of this weight
weight_gradient = float(model.fc.weight.grad[0,0])
print(weight_gradient)

-0.006079344544559717
-0.0056951590813696384


In [14]:
# TODO: manual SGD
lr = 0.01
manually_updated_weight = weight_value_before - lr * weight_gradient

print(f"{weight_value_after:.6f} == {manually_updated_weight:.6f} ?")

-0.006079 == -0.006079 ?


### two models

In [15]:
class MultiLayerCNN(nn.Module):
    def __init__(self):
        super(MultiLayerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 10,
                               kernel_size=5)  # Input is a 3 plane 256x256 tensor (RBB) -> output 10 planes of 254x254
        self.pool_max = nn.MaxPool2d(2, 2)  # output of dim-2 x dim-2
        self.conv2 = nn.Conv2d(10, 20, 5)  # input 10 planes 127x127, output 20 planes of 125x125
        self.pool_avg = nn.AvgPool2d(4, 4)
        self.fc = nn.Linear(20 * 27 * 27, 17)  # single dense layer for the network
        self.batchNorm = nn.BatchNorm2d(3)
        self.loss = nn.BCELoss()
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.batchNorm(x)
        print(f"step 1 : {x.shape}")
        x = self.pool_max(nn.functional.relu(self.conv1(x)))
        print(f"step 2 : {x.shape}")
        x = self.pool_avg(x)
        print(f"step 3 : {x.shape}")
        x = nn.functional.relu(self.conv2(x))
        print(f"step 4 : {x.shape}")
        x = x.view(-1, 20 * 27 * 27)
        print(f"step 5 : {x.shape}")
        x = self.fc(x)
        print(f"step 6 : {x.shape}")
        return x

In [20]:
class CLOUD_MultiLayerCNN(nn.Module):
    def __init__(self):
        super(CLOUD_MultiLayerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 10,
                               kernel_size=10)  # Input is a 3 plane 256x256 tensor (RBB) -> 
                                                # output 10 planes of 218x218
        self.pool_max = nn.MaxPool2d(4, 4)  # output of dim-2 x dim-2
        self.conv2 = nn.Conv2d(10, 20, 5)  # input 10 planes 127x127, output 20 planes of 125x125
        self.pool_avg = nn.AvgPool2d(2, 2)
        self.fc = nn.Linear(13520, 3)  # single dense layer for the network
        self.batchNorm = nn.BatchNorm2d(3)
        self.loss = nn.BCELoss()
        self.sig = nn.Sigmoid()
        self.dropped = nn.Dropout(p=0.1, inplace=False)

    def forward(self, x):
        # x = self.conv1(x)
        # print(f"step 0 : {x.shape}")
        x = self.batchNorm(x)
        print(f"step 1 : {x.shape}")
        x = self.dropped(self.pool_max(nn.functional.relu(self.conv1(x))))
        print(f"step 2 : {x.shape}")
        x = self.pool_avg(x)
        print(f"step 3 : {x.shape}")
        x = self.dropped(nn.functional.relu(self.conv2(x)))
        print(f"step 4 : {x.shape}")
        x = x.view(-1, 20 *26*26)
        print(f"step 5 : {x.shape}")
        x = self.fc(x)
        print(f"step 6 : {x.shape}")
        return x

In [21]:
net = CLOUD_MultiLayerCNN()

net

CLOUD_MultiLayerCNN(
  (conv1): Conv2d(3, 10, kernel_size=(10, 10), stride=(1, 1))
  (pool_max): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool_avg): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=13520, out_features=3, bias=True)
  (batchNorm): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (loss): BCELoss()
  (sig): Sigmoid()
  (dropped): Dropout(p=0.1, inplace=False)
)

In [22]:
net.forward(x)

step 1 : torch.Size([16, 3, 256, 256])
step 2 : torch.Size([16, 10, 61, 61])
step 3 : torch.Size([16, 10, 30, 30])
step 4 : torch.Size([16, 20, 26, 26])
step 5 : torch.Size([16, 13520])
step 6 : torch.Size([16, 3])


tensor([[-0.1976, -0.0446, -0.1900],
        [-0.1295, -0.0140, -0.1165],
        [-0.0932,  0.0017, -0.0851],
        [-0.1577,  0.1086, -0.0597],
        [ 0.0053,  0.0592, -0.1413],
        [-0.0675,  0.0342, -0.0844],
        [-0.1595,  0.0109, -0.2051],
        [-0.2241,  0.0099, -0.1764],
        [-0.1321,  0.0368, -0.1782],
        [-0.0803, -0.0909, -0.1248],
        [-0.2188, -0.0160, -0.2475],
        [-0.1684,  0.0463, -0.1923],
        [-0.2458, -0.0603, -0.1407],
        [-0.2321, -0.0163, -0.0838],
        [-0.1525, -0.0224, -0.0815],
        [-0.1524, -0.0223, -0.1155]], grad_fn=<AddmmBackward0>)

In [23]:
input_size = (16, 3, 256, 256)
summary(net, input_size)

step 1 : torch.Size([16, 3, 256, 256])
step 2 : torch.Size([16, 10, 61, 61])
step 3 : torch.Size([16, 10, 30, 30])
step 4 : torch.Size([16, 20, 26, 26])
step 5 : torch.Size([16, 13520])
step 6 : torch.Size([16, 3])


Layer (type:depth-idx)                   Output Shape              Param #
CLOUD_MultiLayerCNN                      [16, 3]                   --
├─BatchNorm2d: 1-1                       [16, 3, 256, 256]         6
├─Conv2d: 1-2                            [16, 10, 247, 247]        3,010
├─MaxPool2d: 1-3                         [16, 10, 61, 61]          --
├─Dropout: 1-4                           [16, 10, 61, 61]          --
├─AvgPool2d: 1-5                         [16, 10, 30, 30]          --
├─Conv2d: 1-6                            [16, 20, 26, 26]          5,020
├─Dropout: 1-7                           [16, 20, 26, 26]          --
├─Linear: 1-8                            [16, 3]                   40,563
Total params: 48,599
Trainable params: 48,599
Non-trainable params: 0
Total mult-adds (G): 2.99
Input size (MB): 12.58
Forward/backward pass size (MB): 104.99
Params size (MB): 0.19
Estimated Total Size (MB): 117.77